# Obtención de datos

Para obtener los datos vamos a usar las descripciones en cada video de las mañaneras. Pues nos provee de un resumen del video del día. De esta manera ya no tenemos que lidiar con conversaciones del presidente cuando usamos las transcripciones.

## Importaciones

In [63]:
import json 
from datetime import datetime
import re

import pandas as pd 
import numpy as np

from googleapiclient.errors import HttpError
from Google import Create_Service

## Llamamiento a la API de YouTube

<code>retrieve_playlists_items</code> Función que hace la petición a la API de youtube. Se le da un objeto <code>service</code> que es la información sobre el tipo de API y la autenticación. También recibe <code>playlistId</code> que es el ID de la playlist que queremos.

In [64]:
def retrieve_playlists_items(service, playlistId):
    items =[]
    try:
        response = service.playlistItems().list(
            part = 'contentDetails, snippet, status',
            playlistId = playlistId,
            maxResults = 5
        ).execute()

        items.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlistItems().list(
                part = 'contentDetails, snippet, status',
                playlistId = playlistId,
                maxResults = 5,
                pageToken = nextPageToken
            ).execute()

            items.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return items

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Creando constantes para llamar a la API. Toda esta es información sobre el tipo de API y los archivos necesarios para hacer la conexión con Google Cloud

In [65]:
CLIENT_SECRETS_FILE = 'client_secret.json'  # Archivo .json que se obtiene al crear las credenciales OAuth en https://console.cloud.google.com/
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

service = Create_Service(CLIENT_SECRETS_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.readonly'],)
['https://www.googleapis.com/auth/youtube.readonly']
youtube service created successfully


El objeto <code>service</code> nos da accesos a todos los recursos de la API

In [66]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 'abuseReports',
 'activities',
 'captions',
 'channelBanners',
 'channelSections',
 'channels',
 'close',
 'commentThreads',
 'comments',
 'i18nLanguages',
 'i18nRegions',
 'liveBroadcasts',
 'liveChatBans',
 'liveChatMessages',
 'liveChatModerators',
 'liveStreams',
 'members',
 'membershipsLevels',
 'new_batch_h

Hacemos una llamada a a función de <code>retrieve_playlists_items</code> para obtener todos los videos una playlist especifica. En este caso, los videos de la mañaneras

In [67]:
playlistId = 'PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA' # id de la playlist de las mañaneras
playlists_items = retrieve_playlists_items(service, playlistId)

In [68]:
playlists_items[0] # El último video agregado a la playlist

{'kind': 'youtube#playlistItem',
 'etag': 'MjXbq6j_REbkn9jBqTZjh4dAtuE',
 'id': 'UExSbmxSR2FyLV8yOTZLVHNWTDBSNk1FYnB3SnpEOHBwQS42NkYzMkU0NDQ2MTM5NzVE',
 'snippet': {'publishedAt': '2023-05-16T12:45:50Z',
  'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
  'title': 'Gobierno federal atiende demandas del magisterio; diálogo es permanente. Conferencia presidente AMLO',
  'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Martes 16 de mayo 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n16:33 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n21:15 Informe Cero Impunidad\n34:05 Informe de seguridad \n55:23 Informe sobre regularización de vehículos de procedencia extranjera \n58:22 Prevención de adicciones. Cocaína\n1:11:52 Sesión de preguntas y respuestas\n\nCada quince días informamos sobre los daños de las drogas como parte de la atención a las causas de las adicciones. Lo mejor es prevenir el consumo, fortalecer valores, fomentar la unión 

## Guardando los datos en un DataFrame

Los datos están guardados en un json. Podemos obtener cada elemento de la variable utilizando las posiciones.

In [69]:
playlists_items[0]['snippet']

{'publishedAt': '2023-05-16T12:45:50Z',
 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
 'title': 'Gobierno federal atiende demandas del magisterio; diálogo es permanente. Conferencia presidente AMLO',
 'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Martes 16 de mayo 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n16:33 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n21:15 Informe Cero Impunidad\n34:05 Informe de seguridad \n55:23 Informe sobre regularización de vehículos de procedencia extranjera \n58:22 Prevención de adicciones. Cocaína\n1:11:52 Sesión de preguntas y respuestas\n\nCada quince días informamos sobre los daños de las drogas como parte de la atención a las causas de las adicciones. Lo mejor es prevenir el consumo, fortalecer valores, fomentar la unión familiar, que todas y todos tengan oportunidades de trabajo y estudio, acceso al deporte, para no optar por el camino de las conductas antisociales.\xa0\n\nLa cocaína,

Extraeremos sólo la información necesaria para hacer el entrenamiento. Estos datos son: título, descripción y la fecha de publicación

Visualizando los datos que vienen en el <code>.json</code> vemos que la columna <code>'publishedAt'</code> no se refiere a la fecha de publicación del video, sino a la fecha en la que el video se agregó en la playlist. Este es un problema porque los primeros videos tienen todos la misma fecha en esta sección, esto no nos interesa, necesitamos la fecha de publicación del video. Para corregir esto se tiene que llamar de nuevo a la API usando la ID del video que se nos da en el <code>.json</code>. 

In [350]:
data = []

for i, item in enumerate(playlists_items):
    snippet = item['snippet']
    published_at = snippet['publishedAt']
    title = snippet['title']
    description = snippet['description']
    video_id = item['contentDetails']['videoId']

    # Son los últimos 94 son los afectados aquí, así que son los que seleccionamos
    if i >= len(playlists_items) - 94:
        try:
            # Realiza una petición a la API de videos para obtener la fecha de publicación original
            response = service.videos().list(
                part='snippet',
                id=video_id
            ).execute()

            video_info = response.get('items', [])
            if video_info:
                published_at = video_info[0]['snippet']['publishedAt']

        except HttpError as e:
            errMsg = json.loads(e.content)
            print("HTTP Error: ")
            print(errMsg['error']['message'])

    data.append([published_at, title, description])
    
df = pd.DataFrame(data, columns=['published_at', 'title', 'description'])
df = df.head(1101)

print(df)

              published_at                                              title   
0     2023-05-16T12:45:50Z  Gobierno federal atiende demandas del magister...  \
1     2023-05-15T12:44:53Z  Maestras y maestros recibirán aumento de 8.2 p...   
2     2023-05-12T03:54:24Z  Gobierno no permitirá insultos de republicanos...   
3     2023-05-11T12:44:34Z  Sueldos de magistrados y jueces deben bajar pa...   
4     2023-05-10T12:50:23Z  Homenaje a las madres de México en su día. Con...   
...                    ...                                                ...   
1096  2018-12-07T14:42:27Z  Conferencia matutina: Avances del Gabinete de ...   
1097  2018-12-06T14:54:35Z  Hoy anunciamos la terna de candidatos a minist...   
1098  2018-12-05T14:17:13Z  Tercera conferencia de prensa matutina desde P...   
1099  2018-12-04T14:21:56Z     Diálogo con periodistas desde Palacio Nacional   
1100  2018-12-03T15:03:24Z  Primera conferencia de prensa matutina desde P...   

                           

## Una primera limpieza

Los datos que llegan de la API están contaminados con información como minutos de YouTube en formato <code>00:00</code>, links, saltos de línea <code>\n</code> y palabras específicas como nombres redes sociales o frases que no aportan nada.
Para mejor ejemplo, tomemos un elemento.

In [351]:
df['description'][500]

'Conferencia de prensa matutina, desde Palacio Nacional. Miércoles 21 de abril 2021 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n20:03 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n22:25 Informe mensual de seguridad\n1:04:19 Sesión de preguntas y respuestas\n\nEl Gobierno de la Cuarta Transformación trabaja para garantizar la paz en el país. Como cada mes, dimos a conocer las acciones, avances y retos en materia de seguridad pública. \n\nLa percepción de la inseguridad disminuyó y aumentó la confianza de la población en las fuerzas de seguridad. Además, 9 de 11 delitos del fuero federal se mantienen a la baja. Al prevenir la violencia y atender sus causas, impulsamos el desarrollo del país.\n\nDurante el primer trimestre de 2021 se presentaron 4.6 por ciento menos homicidios dolosos respecto al mismo periodo del año anterior. \n\nEn el acumulado de enero a marzo del 2021, Baja California, Chihuahua, Estado de México, Guanajuato, Jalisco y Michoacán 

Declararemos una función <code>replace_patterns</code>que recibirá una row y una serie de patrones que contendrán el regex a buscar y con lo que se va a reeemplazar

In [352]:
def replace_patterns(row, patterns):
    text = row['description']
    for pattern, replacement in patterns.items():
        regex = re.compile(pattern, re.IGNORECASE | re.DOTALL)
        text = regex.sub(replacement, text)
    return text


La función <code>remove_beginning</code> se creó principamente porque el regex estaba dando problemas si no se colocaba en una función, por su complejidad probablemente.

In [353]:
def remove_beginning(string):
    pattern = r'Conferencia\s+de\s+prensa\s+(matutina|en vivo),?(\s+desde\s+\b(\b\w+\b,?\s)?\b\b\w+\b)(,? ([\wáéíóúÁÉÍÓÚ]+\s?)+)?\.\s+\b(Lunes|Martes|Miércoles|Jueves|Viernes|Sábado|Domingo)\b [0-3]?[0-9]\s+de\s+\b(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\b\s+20[0-9][0-9]\.?'
    replace_with = ''
    return re.sub(pattern, replace_with, string)

La función <code>remove_flags</code> este regex también se puso en una función por su complejidad. Este detecta los enlaces de marcade tiempo (<code>00:00</code>)

In [354]:
def remove_flags(string):
    pattern = r'(\d{1,2}:\d{2}(:\d{2})?)\s(.+)'
    replace_with = ''
    return re.sub(pattern, replace_with, string)

La función <code>remove_line</code> este regex quita los saltos de línea <code>\n</code>, es importante que esta fucnión se aplique al final para la limpieza correcta del dataframe

In [355]:
def remove_line(string):
    pattern = r'\n'
    replace_with = ''
    return re.sub(pattern, replace_with, string)

Aquí está los patrones que se buscarán y con lo que se va a reemplazar

In [356]:
patterns = {

    # Borrar separadores
    "\|": "", 
    
    "Presidente AMLO.": "",

    # Borra cualquier link de internet:
    "http(s)?://[a-zA-Z0-9./?=_-]+": "",

    "\s+Más información:": "",

    "lopezobrador": "",

    "Sigue las actividades del presidente de México:": "",
    "Sitio web:\s+": "",
    "YouTube:\s+": "",
    "Facebook:\s+": "",
    "Twitter:\s+": "",
    "Instagram:\s+": "",
    "Telegram:\s+": "",
    "Spotify:": "",

    # Elemina todos los # (hashtags)
    "#\w+": "",

    #Elimina todas las palabras con @ al principio
    "@\w+": "",

    "\w+@\w+.com.mx": "",

    "facebook.com/.org.mx": "",
}   

Aplicamos las funciones y vemos los resultados

In [357]:
df['description'] = df.apply(replace_patterns, patterns=patterns, axis=1).apply(remove_beginning).apply(remove_flags).apply(remove_line)

In [358]:
df['description'][500]

'  El Gobierno de la Cuarta Transformación trabaja para garantizar la paz en el país. Como cada mes, dimos a conocer las acciones, avances y retos en materia de seguridad pública. La percepción de la inseguridad disminuyó y aumentó la confianza de la población en las fuerzas de seguridad. Además, 9 de 11 delitos del fuero federal se mantienen a la baja. Al prevenir la violencia y atender sus causas, impulsamos el desarrollo del país.Durante el primer trimestre de 2021 se presentaron 4.6 por ciento menos homicidios dolosos respecto al mismo periodo del año anterior. En el acumulado de enero a marzo del 2021, Baja California, Chihuahua, Estado de México, Guanajuato, Jalisco y Michoacán concentraron el 50.7 por ciento de las víctimas de este delito. El delito de feminicidio logró contenerse en los primeros tres meses del año al bajar 2.4 por ciento respecto al primer trimestre del año anterior. Los robos se redujeron en 17.8 por ciento en comparación con el primer trimestre de 2020. El se

## Separación por semana

Ahora necesitamos separar por semana, es decir, juntar todas las descripciones de una semana. De esta manera cada renglón del dataframe reprensentará una semana en específico.
Lo primero que debemos hacer es limpiar el apartado de la fecha y convertirlo a datetime.

In [359]:
df['published_at']

0       2023-05-16T12:45:50Z
1       2023-05-15T12:44:53Z
2       2023-05-12T03:54:24Z
3       2023-05-11T12:44:34Z
4       2023-05-10T12:50:23Z
                ...         
1096    2018-12-07T14:42:27Z
1097    2018-12-06T14:54:35Z
1098    2018-12-05T14:17:13Z
1099    2018-12-04T14:21:56Z
1100    2018-12-03T15:03:24Z
Name: published_at, Length: 1101, dtype: object

Lo que se haces es: tomar los primeros 10 digitos de cada elemento de la columna y convertirlo al tipo datetime con la función <code>datetime.strptime</code>. Son los primeros 10 digitos porque son los que representan la fecha de ese día, no nos importan las horas, minutos o segundos así que lo descartamos

In [360]:
clean_dates = []

for dates in df['published_at']:
    clean_dates.append( datetime.strptime(dates[:10], '%Y-%m-%d') )
df['clean_dates'] = clean_dates

In [361]:
df = df.drop('published_at', axis=1) # Ahorra borramos las fechas que no nos sirven

In [362]:
df['clean_dates'] = pd.to_datetime(df['clean_dates'])

Ya tenemos los datos como fechas tipo <code>datetime</code>

In [363]:
df

,title,description,clean_dates
0,Gobierno federal atiende demandas del magister...,Cada quince días informamos sobre los daños ...,2023-05-16
1,Maestras y maestros recibirán aumento de 8.2 p...,Felicitamos a las y los maestros en su día; ...,2023-05-15
2,Gobierno no permitirá insultos de republicanos...,El Gobierno de México mantendrá una política...,2023-05-12
3,Sueldos de magistrados y jueces deben bajar pa...,En cuatro años de Gobierno de la Cuarta Tran...,2023-05-11
4,Homenaje a las madres de México en su día. Con...,Enviamos una cariñosa felicitación a todas l...,2023-05-10
...,...,...,...
1096,Conferencia matutina: Avances del Gabinete de ...,Hoy destacamos la conformación de un sistema...,2018-12-07
1097,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...,2018-12-06
1098,Tercera conferencia de prensa matutina desde P...,Tercera conferencia de prensa matutina desde e...,2018-12-05
1099,Diálogo con periodistas desde Palacio Nacional,Segunda conferencia de prensa matutina del pre...,2018-12-04


In [364]:
df['clean_dates']

0      2023-05-16
1      2023-05-15
2      2023-05-12
3      2023-05-11
4      2023-05-10
          ...    
1096   2018-12-07
1097   2018-12-06
1098   2018-12-05
1099   2018-12-04
1100   2018-12-03
Name: clean_dates, Length: 1101, dtype: datetime64[ns]

La mejor manera de manipular datos con fechas es creando una serie de tiempo, para estos se utiliza la libería <code>datetime</code>. Convertimos el Data Frame en una serie de tiempo con <code>pd.set_index</code>.
De esta manera las fechas ahora serán los índices de el dataframe

In [365]:
df.set_index('clean_dates', inplace=True)

In [366]:
df = df.drop('title', axis=1)

In [367]:
df

,description
clean_dates,
2023-05-16,Cada quince días informamos sobre los daños ...
2023-05-15,Felicitamos a las y los maestros en su día; ...
2023-05-12,El Gobierno de México mantendrá una política...
2023-05-11,En cuatro años de Gobierno de la Cuarta Tran...
2023-05-10,Enviamos una cariñosa felicitación a todas l...
...,...
2018-12-07,Hoy destacamos la conformación de un sistema...
2018-12-06,Cuarta conferencia de prensa matutina desde el...
2018-12-05,Tercera conferencia de prensa matutina desde e...


In [368]:
df['description'].to_csv('dataframe.csv')

In [369]:
df.resample('W').sum()

,description
clean_dates,
2018-12-09,Andrés Manuel López Obrador ofrece su primera ...
2018-12-16,Esta mañana dimos a conocer el trabajo coordin...
2018-12-23,El presupuesto 2019 está planeado para cumplir...
2018-12-30,Celebramos la aprobación del Presupuesto de Eg...
2019-01-06,En la primera conferencia matutina del año pre...
...,...
2023-04-23,En el informe semanal del avance de la const...
2023-04-30,Gracias al pueblo de México por su a...
2023-05-07,En el informe semanal del Tren Maya reafirma...


Ahora le decimos que separe, con respecto a las fechas, semanalmente. De esta manera va a concatenar los contenidos de las descripciones y los títulos.

In [370]:
weekly_ts = df.resample('W').sum()

Se muestra el dataframe

In [371]:
weekly_ts

,description
clean_dates,
2018-12-09,Andrés Manuel López Obrador ofrece su primera ...
2018-12-16,Esta mañana dimos a conocer el trabajo coordin...
2018-12-23,El presupuesto 2019 está planeado para cumplir...
2018-12-30,Celebramos la aprobación del Presupuesto de Eg...
2019-01-06,En la primera conferencia matutina del año pre...
...,...
2023-04-23,En el informe semanal del avance de la const...
2023-04-30,Gracias al pueblo de México por su a...
2023-05-07,En el informe semanal del Tren Maya reafirma...


In [372]:
weekly_ts

,description
clean_dates,
2018-12-09,Andrés Manuel López Obrador ofrece su primera ...
2018-12-16,Esta mañana dimos a conocer el trabajo coordin...
2018-12-23,El presupuesto 2019 está planeado para cumplir...
2018-12-30,Celebramos la aprobación del Presupuesto de Eg...
2019-01-06,En la primera conferencia matutina del año pre...
...,...
2023-04-23,En el informe semanal del avance de la const...
2023-04-30,Gracias al pueblo de México por su a...
2023-05-07,En el informe semanal del Tren Maya reafirma...


Ahora se guarda el dataframe en un .txt para que el preprocesamiento pueda llamarlo y utlizarlo

In [373]:
weekly_ts['description'].to_csv('mañaneras.csv')

Así es como se termina la obtención de datos, ya tenemos nuesto dataframe listo para mandar al preprocesamiento. Como se observó, a los datos se les hizo una primera limpieza, la razón de esto fue para que en el preprocesamiento se hicieran los pasos fundamentales y no pasos extras, como fue este caso de quitar ciertos patrones que estorbaban.